In [0]:
# importando pandas
import pandas as pd
pd.__version__

'0.24.2'

In [0]:
# leitura da base de dados
suicide_rate = pd.read_csv('suicide-rates-overview-1985-to-2016.csv')

# essas colunas possuem valores nulos (HDI for year) e desnecessários (country-year)
suicide_rate = suicide_rate.drop(['country-year', 'HDI for year'], axis=1)

# informações sobre o dataframe
print(suicide_rate.shape)
suicide_rate.info()
suicide_rate.head()

(27820, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 10 columns):
country               27820 non-null object
year                  27820 non-null int64
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
dtypes: float64(1), int64(4), object(5)
memory usage: 2.1+ MB


,country,year,sex,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,"2,156,624,900",796,Boomers


In [0]:
# Agrupando por pais usando groupby
rate_countries = pd.DataFrame(suicide_rate['suicides_no'].groupby(suicide_rate['country']).sum())

# Ordenando pelo número de suicídios
rate_countries = rate_countries.sort_values(by="suicides_no", ascending=False)

# resetando o indice
rate_countries = rate_countries.reset_index()

# atribuindo country como index do dataframe
rate_countries.head(10)

,country,suicides_no
0,Russian Federation,1209742
1,United States,1034013
2,Japan,806902
3,France,329127
4,Ukraine,319950
5,Germany,291262
6,Republic of Korea,261730
7,Brazil,226613
8,Poland,139098
9,United Kingdom,136805


In [0]:
# geocoding library written in Python
!pip install geocoder

     |████████████████████████████████| 102kB 6.0MB/s 


In [0]:
#  importando geocoder
import geocoder
from tqdm import tqdm

In [0]:
# Acessando geojson de cada pais usando geocoder
geojson_mundi = {
  "type": "FeatureCollection",
  "features": []
}

for country in tqdm(rate_countries['country']):
  location = geocoder.arcgis(country).geojson
  location['features'][0]['properties']['name'] = country
  geojson_mundi["features"].append(location["features"][0])
  



  0%|          | 0/101 [00:00<?, ?it/s]

  1%|          | 1/101 [00:00<00:32,  3.12it/s]

  2%|▏         | 2/101 [00:00<00:33,  2.97it/s]

  3%|▎         | 3/101 [00:01<00:33,  2.95it/s]

  4%|▍         | 4/101 [00:01<00:41,  2.35it/s]

  5%|▍         | 5/101 [00:02<00:38,  2.50it/s]



KeyboardInterrupt: ignored

In [0]:
# Install the latest release
!pip install folium==0.8.2

     |████████████████████████████████| 92kB 5.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.2 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [0]:
# importando folium
import folium
folium.__version__

'0.8.2'

In [214]:
# colormap
from branca.colormap import linear

# colormap yellow and green (YlGn)
colormap = linear.YlOrRd_09.scale(
    rate_countries.suicides_no.min(),
    rate_countries.suicides_no.max())

colormap.caption="Suicide in world from 1985 to 2016"

print(colormap(5000.0))

colormap

#ffffcb


In [0]:
# import json
import json

# leitura do geojson do mapa mundi
geojson_mundi = json.load(open('countries.geojson'))

In [184]:
# adicionando os dados no map
filtered_geojson = {
  "type": "FeatureCollection",
  "features": []
}

# adicionando o numero de suicidios ao geojson
data_country = list(rate_countries.country)
aux = []
notin = []
for country in geojson_mundi['features']:
  name_aux = country['properties']['ADMIN']
  if(name_aux in data_country):
    aux.append(name_aux)

# quantos paises estão corretos
print(len(aux))

# pegando os paises com nomes diferentes
for country in data_country:
  if(not(country in aux)):
    notin.append(country)
    
# os que divergirão
print(len(notin))
print(notin)

# com isso podemos fazer uma renomeação antes de realizar a filtragem de fato

101
0
[]


In [185]:
# rename columns
rate_countries.loc[rate_countries.country == "Russian Federation", 'country'] = "Russia"
rate_countries.loc[rate_countries.country == "United States", 'country'] = "United States of America"
rate_countries.loc[rate_countries.country == "Republic of Korea", 'country'] = "South Korea"
rate_countries.loc[rate_countries.country == "Serbia", 'country'] = "Republic of Serbia"
rate_countries.loc[rate_countries.country == "Saint Vincent and Grenadines", 'country'] = "Saint Vincent and the Grenadines"
rate_countries.loc[rate_countries.country == "Bahamas", 'country'] = "The Bahamas"
rate_countries.loc[rate_countries.country == "Cabo Verde", 'country'] = "Cape Verde"
rate_countries.loc[rate_countries.country == "Macau", 'country'] = "Macao S.A.R"

# reindex the dataframe
rate_countries.set_index('country', inplace=True)
rate_countries.index.name = None
rate_countries['country'] = rate_countries.index
rate_countries.head(10)

,suicides_no,country
Russia,1209742,Russia
United States of America,1034013,United States of America
Japan,806902,Japan
France,329127,France
Ukraine,319950,Ukraine
Germany,291262,Germany
South Korea,261730,South Korea
Brazil,226613,Brazil
Poland,139098,Poland
United Kingdom,136805,United Kingdom


In [186]:
# filtrando os paises presentes no dataset original
for country in geojson_mundi['features']:
  name_aux = country['properties']['ADMIN']
  if(name_aux in data_country):
    country['properties']['suicides_no'] = str(rate_countries.loc[name_aux, 'suicides_no'])
    filtered_geojson['features'].append(country)

len(filtered_geojson['features'])

101

In [0]:
from folium.plugins import MiniMap

In [201]:
# Criando mapa com folium e colomap
mp = folium.Map(
  location=[30,0],
  tiles='openstreetmap',
  zoom_start=2
)
  
folium.GeoJson(filtered_geojson,
              name='Number of suicides',
              style_function=lambda country: {'fillColor': colormap(rate_countries.loc[country['properties']['ADMIN'], 'suicides_no']),
                                              'color': 'black',
                                              'weight': 2,
                                              'fillOpacity':0.8},
              tooltip=folium.GeoJsonTooltip(fields=['ADMIN', "suicides_no"],
                                           aliases=['Country:','Nº Suicides:'],
                                           localize=True)
              ).add_to(mp)  


# create and add a minimap
minimap = MiniMap()
mp.add_child(minimap)

# layer control
folium.LayerControl().add_to(mp)

# adicionando o colormap
colormap.add_to(mp)

mp.save('suicide.html')
# show the map
mp

Output hidden; open in https://colab.research.google.com to view.

In [0]:
from google.colab import files
files.download('suicide.html') 